# **ETAPA 5 - PROCESSAMENTO PARA CAMADA GOLD**

<br>

---

<br>

Essa etapa será responsável por mover os dados para a camada gold, agrupando os dados. 

<br>

*`Esse é um modelo modular, complete as informações necessárias nos trechos que estão destacados em vermelho assim como esse, seguindo o padrão snake_case.`*

<br> 

***AVISO**: Esse Notebook foi feito com base na estrutura do Databricks Free Edition, que utiliza catálogos.*

<br><br>

---

<br>

### Parte 1 - **Importação das Bibliotecas Necessárias**

In [0]:
# import pyspark

from os import listdir
import uuid
import gc
import urllib.request
from pyspark.sql.functions import *
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, TimestampType, LongType, DateType, TimestampType, BooleanType, DoubleType

# Gold Layer

## Configuração do servidor para trabalho no Gold Layer

In [0]:

# Create a SparkSession with the required configurations for Delta Lake

spark = SparkSession.builder \
    .appName("Carga Delta") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()


## Funções

### Funções para verificar o id_job e caminhos.
Com execeção dos paths dentro do schema gold, qualquer erro levantado nas funções abaixo é com relação a processos de outras camadas

In [0]:
# verifica se os caminhos para a camada gold existem
def gold_volume():
    try:
        spark.sql("create schema if not exists dataexperts.gold")
        if not spark.sql("show volumes in dataexperts.gold").collect():
            spark.sql(f"create volume if not exists dataexperts.gold.main")
            print("Volume criado com sucesso.")
            return 1
        else:
            print("Volume já existe.")
            return 1
    except Exception as e:
        print(f"Ocorreu um erro na função gol_directories(): {e}") 
        return 0

# verifica se os caminhos para a camada silver existem
def exist_silver_path(volume):
    volume = volume.lower()
    try:
        command = spark.sql(f"show volumes in dataexperts.silver").collect() 
        if not command:  
            print(f"Não existem volumes na camada silver")
            return 0
        else:
            if command[0][1] == volume:
                print(f"O volume {volume} existe na camada silver")
                return 1
            print(f"O volume {volume} não existe na camada silver data_ex.silver")
            return 0
    except Exception as e:
        print(f"Ocorreu um erro na função exist_silver_path(): {e})")
        return 0


### (df || 0) read(path, volume)

In [0]:
def read(volume, file_name):
    try:
        df = spark.read\
            .format("delta")\
            .load(f"/Volumes/dataexperts/silver/{volume}/{file_name}")
        return df
    except Exception as e:
        print(f"Erro ao tentar ler {file_name}: {e}")
        return 0

### (df || 0) save(df, new_file_name)

In [0]:
def save_table(df, new_file_name):
    try:
        if "fato" in new_file_name:
            df_data = spark.read.format("delta").load("/Volumes/dataexperts/gold/main/dim_data")
            
            df_fato = df\
                .join(broadcast(
                    df_data.select("sk_data", "ano", "mes")),
                      "sk_data")\
            
            df_fato\
                .write\
                .format("delta")\
                .mode("overwrite")\
                .partitionBy("ano","mes")\
                .saveAsTable(f"dataexperts.gold.{new_file_name}")
            
            df_fato\
                .write\
                .format("delta")\
                .mode("overwrite")\
                .save(f"/Volumes/dataexperts/gold/main/{new_file_name}")
            
            return df_fato
        else:
            df.write\
                .format("delta")\
                .mode("overwrite")\
                .saveAsTable(f"dataexperts.gold.{new_file_name}")
            df.write\
                .format("delta")\
                .mode("overwrite")\
                .save(f"/Volumes/dataexperts/gold/main/{new_file_name}")
            return df
    except Exception as e:
        print(f"Ocorreu um erro na função save(): {e}")
        return 0

### execute_gold_layer()

In [0]:
# Lista de logs
log_data = {}

def execute_gold_layer(volume, camada, new_files_names):

    # Garante que as pastas no unity catalog gold existam
    if not gold_volume():
        return

    # Verifica se existe o caminho até a camada silver
    if not exist_silver_path(volume):
        return
    
    files = listdir("/Volumes/dataexperts/silver/vendas")
    
    for i, (file, new_file_name) in enumerate(zip(files,new_files_names)): 

        # Ler o arquivo
        df_read = read(volume, file)
        
        # df_read e nulo ou 0
        if not df_read:
            return

        # Salvar o df_read na camada gold
        df_read = save_table(df_read, new_file_name)

        # df_read e nulo ou 0
        if not df_read:
            return

        print(f"Arquivo {file} processado com sucesso.")

        
        


# Main

In [0]:
volume = "vendas"
camada = "gold"

new_files_names = [
    "dim_categoria_produto",
    "dim_cliente",
    "dim_data", 
    "dim_localidade",
    "dim_produto",
    "fato_vendas"
] 

execute_gold_layer(volume, camada, new_files_names)

# Impressão dos resultados

In [0]:
%skip

volume = "download001"
camada = "gold"

new_files_names = [
    "gold_dim_categoria_produto",
    "gold_dim_cliente",
    "gold_dim_data", 
    "gold_dim_localidade",
    "gold_dim_produto",
    "gold_fato_vendas"
] 

def see_tables(camada, new_file_names):
    for file in new_files_names:
        df = spark.read.format("delta").load(f"/Volumes/data_ex/{camada}/main/{file}")
        display(df)

see_tables("gold", new_files_names)


# Salvando como table para exportação

# Limpar Memória

In [0]:
import gc

gc.collect()

# Reset

In [0]:
# spark.sql("drop schema if exists data_ex.gold cascade")